# Hotel Booking Demand

## Business Problem Understanding

**Context**

In the hospitality industry, efficient management of hotel reservations is necessary to optimize revenue. Booking cancellations could potentially disrupt operational schedules and reduce potential revenues. Being able to predict booking cancellations is important for implementing strategies to reduce its impact on scheduling and operations.

**Problem Statement**

Booking cancellation can cause potential revenue loss, underutilization of hotel assets, and operational schedule disruption. It is difficult to predict cancellation in advance. Therefore, it is necessary to develop a solution that could predict booking cancellations in advance to minimize potential loss and disruptions.

**Goals**

The goal is to develop a model that could accurately predict booking cancellations before they occur. This model uses historical customer booking data to identify potential cancellations. This could help the hotel in implementing proactive measures to the particular booking such as dynamic pricing or discount offers to discourage cancellation or mitigate potential revenue loss.

**Analytic Approach**

The approach involves preprocessing the data, selecting relevant features (such as previous cancellations, booking changes), and training machine learning models (such as logistic regression, decision trees, or ensemble methods) to predict the probability of a booking being canceled. 

**Metric Evaluation**

The performance of the models will be evaluated using metrics such as accuracy, precision, recall, and F1-score. These metrics will assess the model's ability to correctly identify bookings as either canceled or not canceled. For this case, the evaluation metric is focusing on the recall scores of the cancelled bookings.

## Data Understanding

- This data set contains booking information for a hotel located in Portugal, and includes information regarding room reservation for respective customers.
- All personally identifying information has been removed from the data.


**Features**

-	*country*: Country of origin.
-	*market_segment*: Market segment designation. 
-	*previous_cancellations*: Number of previous bookings that were cancelled by the customer prior to the current booking.
-	*booking_changes*: Number of changes/amendments made to the booking from the moment the booking was entered on the PMS until the moment of check-in or cancellation.
-	*deposit_type*: Indication on if the customer made a deposit to guarantee the booking. 
-	*days_in_waiting_list*: Number of days the booking was in the waiting list before it was confirmed to the customer.
-	*customer_type*: Type of booking.
-	*reserved_room_type*: Code of room type reserved. Code is presented instead of designation for anonymity reasons.
-	*required_car_parking_space*: Number of car parking spaces required by the customer.
-	*total_of_special_request*: Number of special requests made by the customer (e.g. twin bed or high floor).
-	*is_canceled*: Value indicating if the booking was canceled (1) or not (0).


## Data Cleaning & Preprocessing

In [108]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.metrics import make_scorer, recall_score


from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import chi2
from scipy.stats import pointbiserialr

from sklearn.model_selection import GridSearchCV

In [109]:
df = pd.read_csv('data_hotel_booking_demand.csv')
df

,country,market_segment,previous_cancellations,booking_changes,deposit_type,days_in_waiting_list,customer_type,reserved_room_type,required_car_parking_spaces,total_of_special_requests,is_canceled
0,IRL,Offline TA/TO,0,0,No Deposit,0,Transient-Party,A,0,0,0
1,FRA,Online TA,0,0,No Deposit,0,Transient,A,0,2,0
2,PRT,Online TA,0,1,No Deposit,0,Transient,A,0,2,0
3,NLD,Online TA,0,0,No Deposit,0,Transient,A,0,1,1
4,PRT,Online TA,0,2,No Deposit,0,Transient,A,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...
83568,PRT,Online TA,0,0,No Deposit,0,Contract,A,0,3,1
83569,FRA,Online TA,0,0,No Deposit,0,Transient,D,0,2,0
83570,PRT,Online TA,0,1,No Deposit,0,Transient,D,0,3,0
83571,CN,Offline TA/TO,0,0,No Deposit,0,Transient-Party,A,0,0,0


In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83573 entries, 0 to 83572
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   country                      83222 non-null  object
 1   market_segment               83573 non-null  object
 2   previous_cancellations       83573 non-null  int64 
 3   booking_changes              83573 non-null  int64 
 4   deposit_type                 83573 non-null  object
 5   days_in_waiting_list         83573 non-null  int64 
 6   customer_type                83573 non-null  object
 7   reserved_room_type           83573 non-null  object
 8   required_car_parking_spaces  83573 non-null  int64 
 9   total_of_special_requests    83573 non-null  int64 
 10  is_canceled                  83573 non-null  int64 
dtypes: int64(6), object(5)
memory usage: 7.0+ MB


In [111]:
list_item = []
for col in df.columns:
    list_item.append([col,df[col].nunique(),df[col].unique()])

tabel_desc = pd.DataFrame(columns=['Column Name', 'Number of Unique', 'Unique Sample'],
                     data=list_item)
pd.set_option('display.max_colwidth', 0)
tabel_desc

,Column Name,Number of Unique,Unique Sample
0,country,162,"[IRL, FRA, PRT, NLD, ESP, UMI, CN, LUX, BRA, BEL, JPN, DEU, ITA, CHE, GBR, AGO, SRB, COL, CHN, SWE, AUT, CIV, CZE, POL, USA, SGP, RUS, ROU, DNK, IND, MAR, PHL, ARG, ISL, ZAF, LBN, MOZ, TUR, BGD, MEX, CAF, NOR, FIN, UKR, EGY, ISR, nan, KOR, AZE, HUN, AUS, EST, CHL, SVN, PRY, ABW, ALB, LTU, ARE, HRV, SAU, NZL, LVA, ATA, KAZ, DZA, TWN, CRI, BIH, BGR, IRQ, OMN, VEN, IDN, GEO, MLT, IRN, BLR, URY, LBY, TUN, BEN, MYS, MWI, GRC, CYP, CPV, HKG, PRI, MKD, MUS, IMN, PAN, NGA, GLP, KHM, PER, QAT, SEN, MAC, ...]"
1,market_segment,8,"[Offline TA/TO, Online TA, Direct, Groups, Corporate, Complementary, Aviation, Undefined]"
2,previous_cancellations,15,"[0, 1, 25, 2, 3, 11, 24, 13, 6, 26, 14, 4, 19, 5, 21]"
3,booking_changes,19,"[0, 1, 2, 5, 3, 4, 6, 7, 9, 8, 17, 10, 13, 12, 14, 21, 16, 20, 15]"
4,deposit_type,3,"[No Deposit, Non Refund, Refundable]"
5,days_in_waiting_list,115,"[0, 3, 39, 87, 77, 215, 97, 162, 46, 79, 4, 31, 122, 34, 57, 259, 50, 17, 32, 43, 75, 69, 44, 22, 113, 58, 38, 65, 48, 236, 176, 25, 108, 91, 19, 160, 68, 28, 63, 35, 42, 59, 24, 8, 183, 187, 21, 111, 56, 98, 101, 15, 16, 47, 93, 45, 14, 70, 10, 49, 178, 99, 60, 40, 85, 223, 6, 33, 391, 20, 1, 41, 147, 55, 80, 379, 150, 61, 330, 174, 107, 96, 18, 62, 13, 30, 120, 27, 207, 71, 2, 76, 125, 5, 9, 224, 53, 83, 84, 12, ...]"
6,customer_type,4,"[Transient-Party, Transient, Contract, Group]"
7,reserved_room_type,10,"[A, E, D, F, B, G, C, H, L, P]"
8,required_car_parking_spaces,5,"[0, 1, 2, 8, 3]"
9,total_of_special_requests,6,"[0, 2, 1, 3, 4, 5]"


In [112]:
df_prep = df.copy()

Missing Values

In [113]:
df_prep.isna().sum().sort_values(ascending = False)

country                        351
market_segment                 0  
previous_cancellations         0  
booking_changes                0  
deposit_type                   0  
days_in_waiting_list           0  
customer_type                  0  
reserved_room_type             0  
required_car_parking_spaces    0  
total_of_special_requests      0  
is_canceled                    0  
dtype: int64

Percentage of Missing Values

In [114]:
# Percentage of Missing Values
(df_prep.isna().sum()/df_prep.shape[0]*100).sort_values(ascending = False)

country                        0.419992
market_segment                 0.000000
previous_cancellations         0.000000
booking_changes                0.000000
deposit_type                   0.000000
days_in_waiting_list           0.000000
customer_type                  0.000000
reserved_room_type             0.000000
required_car_parking_spaces    0.000000
total_of_special_requests      0.000000
is_canceled                    0.000000
dtype: float64

Missing Values of market_segment columns in percentage

In [115]:
(df_prep['market_segment'].value_counts()['Undefined']/df_prep['market_segment'].shape[0]*100)

0.0011965586971868904

Because the missing values on both market_segment and country columns are low (under 1%), it could be dropped from further analysis.

In [116]:
df_prep = df_prep.dropna()

In [117]:
df_prep = df_prep.drop(df_prep[df_prep['market_segment'] == 'Undefined'].index)

In [118]:
list_item = []
for col in df_prep.columns:
    list_item.append([col,df_prep[col].nunique(),df_prep[col].unique()])

tabel_desc = pd.DataFrame(columns=['Column Name', 'Number of Unique', 'Unique Sample'],
                     data=list_item)
pd.set_option('display.max_colwidth', 0)
tabel_desc

,Column Name,Number of Unique,Unique Sample
0,country,162,"[IRL, FRA, PRT, NLD, ESP, UMI, CN, LUX, BRA, BEL, JPN, DEU, ITA, CHE, GBR, AGO, SRB, COL, CHN, SWE, AUT, CIV, CZE, POL, USA, SGP, RUS, ROU, DNK, IND, MAR, PHL, ARG, ISL, ZAF, LBN, MOZ, TUR, BGD, MEX, CAF, NOR, FIN, UKR, EGY, ISR, KOR, AZE, HUN, AUS, EST, CHL, SVN, PRY, ABW, ALB, LTU, ARE, HRV, SAU, NZL, LVA, ATA, KAZ, DZA, TWN, CRI, BIH, BGR, IRQ, OMN, VEN, IDN, GEO, MLT, IRN, BLR, URY, LBY, TUN, BEN, MYS, MWI, GRC, CYP, CPV, HKG, PRI, MKD, MUS, IMN, PAN, NGA, GLP, KHM, PER, QAT, SEN, MAC, SVK, ...]"
1,market_segment,7,"[Offline TA/TO, Online TA, Direct, Groups, Corporate, Complementary, Aviation]"
2,previous_cancellations,15,"[0, 1, 25, 2, 3, 11, 24, 13, 6, 26, 14, 4, 19, 5, 21]"
3,booking_changes,19,"[0, 1, 2, 5, 3, 4, 6, 7, 9, 8, 17, 10, 13, 12, 14, 21, 16, 20, 15]"
4,deposit_type,3,"[No Deposit, Non Refund, Refundable]"
5,days_in_waiting_list,115,"[0, 3, 39, 87, 77, 215, 97, 162, 46, 79, 4, 31, 122, 34, 57, 259, 50, 17, 32, 43, 75, 69, 44, 22, 113, 58, 38, 65, 48, 236, 176, 25, 108, 91, 19, 160, 68, 28, 63, 35, 42, 59, 24, 8, 183, 187, 21, 111, 56, 98, 101, 15, 16, 47, 93, 45, 14, 70, 10, 49, 178, 99, 60, 40, 85, 223, 6, 33, 391, 20, 1, 41, 147, 55, 80, 379, 150, 61, 330, 174, 107, 96, 18, 62, 13, 30, 120, 27, 207, 71, 2, 76, 125, 5, 9, 224, 53, 83, 84, 12, ...]"
6,customer_type,4,"[Transient-Party, Transient, Contract, Group]"
7,reserved_room_type,10,"[A, E, D, F, B, G, C, H, L, P]"
8,required_car_parking_spaces,5,"[0, 1, 2, 8, 3]"
9,total_of_special_requests,6,"[0, 2, 1, 3, 4, 5]"


In [119]:
df_prep.isna().sum().sort_values(ascending = False)

country                        0
market_segment                 0
previous_cancellations         0
booking_changes                0
deposit_type                   0
days_in_waiting_list           0
customer_type                  0
reserved_room_type             0
required_car_parking_spaces    0
total_of_special_requests      0
is_canceled                    0
dtype: int64

Remove Duplicates

In [120]:
df_prep.duplicated().sum()

73138

In [121]:
df_prep.drop_duplicates(inplace=True)

In [122]:
df_prep.duplicated().sum()

0

In [123]:
df_prep

,country,market_segment,previous_cancellations,booking_changes,deposit_type,days_in_waiting_list,customer_type,reserved_room_type,required_car_parking_spaces,total_of_special_requests,is_canceled
0,IRL,Offline TA/TO,0,0,No Deposit,0,Transient-Party,A,0,0,0
1,FRA,Online TA,0,0,No Deposit,0,Transient,A,0,2,0
2,PRT,Online TA,0,1,No Deposit,0,Transient,A,0,2,0
3,NLD,Online TA,0,0,No Deposit,0,Transient,A,0,1,1
4,PRT,Online TA,0,2,No Deposit,0,Transient,A,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...
83515,GBR,Direct,0,2,No Deposit,0,Transient-Party,A,1,1,0
83543,CHN,Online TA,0,0,No Deposit,0,Transient-Party,C,0,2,0
83554,BEL,Groups,0,0,No Deposit,0,Transient-Party,A,0,1,0
83562,NLD,Online TA,0,0,No Deposit,0,Transient,B,0,0,0


Check for Imbalance datasets

In [124]:
df_prep['is_canceled'].value_counts(normalize=True)

is_canceled
0    0.762471
1    0.237529
Name: proportion, dtype: float64

### Using Chi-Squared method to remove irrelevant categorical features (independent variables with p-values more than 0.05)

In [125]:
categorical_features = df_prep.select_dtypes(include=['object']).columns.tolist()
categorical_features

['country',
 'market_segment',
 'deposit_type',
 'customer_type',
 'reserved_room_type']

In [126]:
df_check = df_prep.select_dtypes(include=['object']).copy()

for col in categorical_features:
    le = LabelEncoder()
    df_check[col] = le.fit_transform(df_check[col])
df_check

,country,market_segment,deposit_type,customer_type,reserved_room_type
0,69,5,0,3,0
1,51,6,0,2,0
2,123,6,0,2,0
3,112,6,0,2,0
4,123,6,0,2,0
...,...,...,...,...,...
83515,54,3,0,3,0
83543,28,6,0,3,2
83554,13,4,0,3,0
83562,112,6,0,2,1


In [127]:
features = df_check
target = df_prep['is_canceled']


In [128]:
chi_scores = chi2(features, target)
chi_scores

(array([2353.40407654,   85.98684316,  337.05549318,    3.9771342 ,
           4.58023576]),
 array([0.00000000e+00, 1.81136785e-20, 2.79344349e-75, 4.61219707e-02,
        3.23427647e-02]))

The p-values of all the columns tested above are lower than 0.05, which means the null hypothesis of independent variables is rejected. Therefore, all features above will be included in the modelling process.

### Using Point-biserial correlation coefficient to remove irrelevant continuous features

In [ ]:
features = df_prep.select_dtypes(exclude=['object']).drop('is_canceled', axis=1).copy()
target = df_prep['is_canceled']

In [ ]:
features.columns.tolist()

['previous_cancellations',
 'booking_changes',
 'days_in_waiting_list',
 'required_car_parking_spaces',
 'total_of_special_requests']

In [ ]:
for i in features:
    correlation, p_value = pointbiserialr(features[i], target)
    print(f'Point-Biserial Correlation: {correlation}, p-value: {p_value}')


Point-Biserial Correlation: 0.07670020847910905, p-value: 1.2367393004097703e-14
Point-Biserial Correlation: -0.07272954441731139, p-value: 2.634820561903737e-13
Point-Biserial Correlation: 0.03222238110295828, p-value: 0.001212081009330368
Point-Biserial Correlation: -0.2351864236431272, p-value: 9.283062651065228e-127
Point-Biserial Correlation: -0.07322334791724212, p-value: 1.816761745926394e-13


The p-values of all the columns tested above are lower than 0.05, which means the null hypothesis of independent variables is rejected. Therefore, all features above will be included in the modelling process.

In [ ]:

categorical_features = df_prep.select_dtypes(include=['object']).copy().columns.tolist()
numerical_features = df_prep.select_dtypes(exclude=['object']).drop('is_canceled', axis=1).copy().columns.tolist()


In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])


For the categorical features, OneHotEncoder is used to convert categorical variables into a binary format, where each category becomes a separate binary feature. By encoding categorical variables as binary features, the model's predictions remain interpretable, as each feature represents the presence or absence of a specific category.  This ensures that all categories are adequately represented without introducing bias or assuming an ordinal relationship.

For the numerical features, StandardScaler standardizes numerical features by removing the mean and scaling them to unit variance. This transformation ensures that numerical features have a similar scale, preventing features with larger values from dominating those with smaller values during model training. Compared to other method, for example MinMaxScaler that maps the data to a fixed range, which may not be suitable for features with small variance, StandardScaler, which centers the data around zero and scales it to unit variance, may provide better results.

In [ ]:
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, class_weight={0:1, 1:4}))
])


RandomForestClassifier algorithm works by creating several subsets of the training data with bootstrap sampling. FOr each sample, then a decision tree is trained independently. During training, a random subset of features is selected at each split point. Once all decision trees are trained, the algorithm then makes predictions by aggregating predictions from individual trees.

The advantage of RandomForestClassifier is that it can handle both numerical and categorical features without extensive preprocessing which makes it versatile, being able to accommodate various input data types without complex feature engineering. In real-world datasets where features may have different data types, such as booking amounts (numeric) and booking channels (categorical), the algorithm simplifies the modeling process and accelerates deployment which facilitates faster decision-making.

RandomForestClassifier also can capture complex non-linear relationships between features and the target variable by constructing multiple decision trees and aggregating their predictions. This makes it suitable for datasets with certain interactions among variables, such as predicting order cancellations influenced by various factors. In such conditions, the algorithm can provide more accurate predictions compared to linear models like Logistic Regression.


## Modelling

In [ ]:
X = df_prep.drop('is_canceled', axis=1)
y = df_prep[['is_canceled']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [ ]:
model.fit(X_train, y_train)

c:\Users\gpram\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['previous_cancellations',
                                                   'booking_changes',
                                                   'days_in_waiting_list',
                                                   'required_car_parking_spaces',
                                                   'total_of_special_requests']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['country', 'market_segment',
                                                   'deposit_type',
                                                   'customer_type',
                                                   'reserved_room_type'])])),
                ('classifier',
                 RandomForestClassifier(class_weight={0: 1, 1: 4},
                                        random_state=42))])

In [ ]:
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.80      0.79      0.80      2306
           1       0.36      0.36      0.36       719

    accuracy                           0.69      3025
   macro avg       0.58      0.58      0.58      3025
weighted avg       0.69      0.69      0.69      3025

ROC AUC Score: 0.7220026489523008


## Hyperparameter Tuning

In [ ]:
param_grid = {
    'classifier__n_estimators': [100, 300, 500],
    'classifier__min_samples_split': [2, 5, 10, 20, 30]
}

recall_scorer = make_scorer(recall_score, pos_label=1)
grid_search = GridSearchCV(model, param_grid, cv=5, scoring=recall_scorer, n_jobs=-1)
grid_search.fit(X_train, y_train)

print(f'Best parameters: {grid_search.best_params_}')



c:\Users\gpram\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Best parameters: {'classifier__min_samples_split': 30, 'classifier__n_estimators': 100}


In [ ]:
model.set_params(**grid_search.best_params_)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred))

c:\Users\gpram\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


              precision    recall  f1-score   support

           0       0.92      0.69      0.79      2306
           1       0.45      0.82      0.58       719

    accuracy                           0.72      3025
   macro avg       0.69      0.75      0.69      3025
weighted avg       0.81      0.72      0.74      3025



## Conclusion

In conclusion, the machine learning model for predicting hotel cancellations demonstrated a high recall rate of 0.82 score in identifying a significant proportion of actual cancellation.

The business impact of the model is as follows :
For example, if 
the total number of bookings are 3025, 
the baseline cancellation rate is 30%, with revenue per booking at $100

Without Model Prediction 

3025 x (1-0.3) x $100 = $211700

With Model Prediciton 
If all the previously potentially cancelled booking predicted successfuly and willing to accept a 50% discount, then
(3025 x (1-0.3) x $100) + (0.3 x 3025 x $50) = $257075

Percentage increase
$45375/$211700 = 21,4% increase in revenue

### Limitation

The model treats all time periods equally. However, cancellation patterns may vary seasonally, during holidays, or in response to external events. This could result in inaccurate predictions during peak cancellation periods or during certain emerging trends and patterns over time. Incorporating time-dependent features or time series analysis techniques could enhance the model's predictive accuracy.

The current model may be based on a limited set of input features, potentially not including important predictors of booking cancellations. Features such as customer preferences, booking history, or external factors like economic conditions or travel restrictions may not be recorded. This could lead to poor performance and incomplete understanding of reasons for booking cancellation.

The current model also have limitation to only be able to predict with good performance if the test data is already within the range of the trained dataset.

- country: 162 countries in training dataset
- market_segment: Offline TA/TO, Online TA, Direct, Groups, Corporate, Complementary, Aviation		
- previous_cancellations: 0 - 26
- booking_changes:  0-21	
- deposit_type:  No Deposit, Non Refund, Refundable	
- days_in_waiting_list: 0-391	
- customer_type: Transient-Party, Transient, Contract, Group
- reserved_room_type: A, E, D, F, B, G, C, H, L, P
- required_car_parking_space: 0-8	
- total_of_special_request: 0-5


### Recommendation

Establish a schedule for periodically retraining the cancellation prediction model on fresh data to capture evolving patterns and trends in booking cancellations.

Perform periodic checks to ensure the quality of  future input data sources. Address any inconsistencies, missing values, or data anomalies to maintain the reliability of the model's predictions.

Continuously assess the relevance and effectiveness of input features. Introduce new features or remove irrelevant ones based on current conditions and insights from data analysis.

In [130]:
import pickle
with open('trained_model.pkl', 'wb') as file:
    pickle.dump(model, file)